In [1]:
import pandas as pd
import requests
import time
import numpy as np
import json
import concurrent.futures
import jsonlines


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
challenger_match_hists = pd.read_csv('../data/challenger_match_hists.csv')

In [2]:
region_base_url_dict = {'na':'na1.api.riotgames.com',
          'br':'br1.api.riotgames.com',
          'eun':'eun1.api.riotgames.com',
          'euw':'euw1.api.riotgames.com',
          'jp':'jp1.api.riotgames.com',
          'kr':'kr.api.riotgames.com',
          'la1':'la1.api.riotgames.com',
          'la2':'la2.api.riotgames.com',
          'oce':'oc1.api.riotgames.com',
          'tr':'tr1.api.riotgames.com',
          'ru':'ru.api.riotgames.com'}
            

In [10]:
api_key = pd.read_json('../secrets.json')['riot_api_key'][0]

timeline_by_match_id_url = '/lol/match/v4/timelines/by-match/'

match_by_match_id_url = '/lol/match/v4/matches/'

account_by_name_url = '/lol/summoner/v4/summoners/by-name/'

match_hist_by_id_url = '/lol/match/v4/matchlists/by-account/'

challenger_ladder_url = '/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'

summoner_by_summoner_id = '/lol/summoner/v4/summoners/'

In [39]:
champions = requests.get('http://ddragon.leagueoflegends.com/cdn/10.16.1/data/en_US/champion.json').json()

In [40]:
champions_df = pd.DataFrame(champions['data']).transpose()

In [55]:
champions_df.to_csv('../../Capstone_app/data/champions_df.csv',index=False)

In [50]:
champions_df[champions_df['name'] == 'Elise']

,version,id,key,name,title,blurb,info,image,tags,partype,stats
Elise,10.16.1,Elise,60,Elise,the Spider Queen,Elise is a deadly predator who dwells in a shu...,"{'attack': 6, 'defense': 5, 'magic': 7, 'diffi...","{'full': 'Elise.png', 'sprite': 'champion0.png...","[Mage, Fighter]",Mana,"{'hp': 534, 'hpperlevel': 93, 'mp': 324, 'mppe..."


In [54]:
print(champions_df['name'].values)

['Aatrox' 'Ahri' 'Akali' 'Alistar' 'Amumu' 'Anivia' 'Annie' 'Aphelios'
 'Ashe' 'Aurelion Sol' 'Azir' 'Bard' 'Blitzcrank' 'Brand' 'Braum'
 'Caitlyn' 'Camille' 'Cassiopeia' "Cho'Gath" 'Corki' 'Darius' 'Diana'
 'Draven' 'Dr. Mundo' 'Ekko' 'Elise' 'Evelynn' 'Ezreal' 'Fiddlesticks'
 'Fiora' 'Fizz' 'Galio' 'Gangplank' 'Garen' 'Gnar' 'Gragas' 'Graves'
 'Hecarim' 'Heimerdinger' 'Illaoi' 'Irelia' 'Ivern' 'Janna' 'Jarvan IV'
 'Jax' 'Jayce' 'Jhin' 'Jinx' "Kai'Sa" 'Kalista' 'Karma' 'Karthus'
 'Kassadin' 'Katarina' 'Kayle' 'Kayn' 'Kennen' "Kha'Zix" 'Kindred' 'Kled'
 "Kog'Maw" 'LeBlanc' 'Lee Sin' 'Leona' 'Lillia' 'Lissandra' 'Lucian'
 'Lulu' 'Lux' 'Malphite' 'Malzahar' 'Maokai' 'Master Yi' 'Miss Fortune'
 'Wukong' 'Mordekaiser' 'Morgana' 'Nami' 'Nasus' 'Nautilus' 'Neeko'
 'Nidalee' 'Nocturne' 'Nunu & Willump' 'Olaf' 'Orianna' 'Ornn' 'Pantheon'
 'Poppy' 'Pyke' 'Qiyana' 'Quinn' 'Rakan' 'Rammus' "Rek'Sai" 'Renekton'
 'Rengar' 'Riven' 'Rumble' 'Ryze' 'Sejuani' 'Senna' 'Sett' 'Shaco' 'Shen'
 'Shyvana' 'S

In [ ]:
#challenger_ladder = requests.get(f'https://na1.api.riotgames.com{challenger_ladder_url}?api_key={api_key}')

In [ ]:
#challenger_ladder.json()

In [ ]:
def get_all_challengers():
    challengers_df = pd.DataFrame()
    for key, base_url in region_base_url_dict.items():
        ladder_response = requests.get(f'https://{base_url}{challenger_ladder_url}?api_key={api_key}')
        try:
            assert(ladder_response.status_code == 200)
            response_df = pd.DataFrame(ladder_response.json()['entries'])
            response_df['region'] = [key] * len(response_df)
            challengers_df = pd.concat([challengers_df,response_df])
        except:
            print(f'Bad request for {key}: {ladder_response.status_code}')
        time.sleep(1.2001)
        
    return challengers_df

In [8]:
def get_summoner_by_name(base_url, summoner_name):
    return requests.get(f'https://{base_url}{account_by_name_url}{summoner_name}?api_key={api_key}')

In [4]:
def get_summoner(base_url, summoner_id):
    return requests.get(f'https://{base_url}{summoner_by_summoner_id}{summoner_id}?api_key={api_key}')

In [5]:
def get_match_hist(account_id, base_url, queue = '420'):
    return requests.get(f'https://{base_url}{match_hist_by_id_url}{account_id}?api_key={api_key}&queue={queue}')

In [6]:
def get_match(match_id, base_url):
    return requests.get(f'https://{base_url}{match_by_match_id_url}{match_id}?api_key={api_key}'),requests.get(f'https://{base_url}{timeline_by_match_id_url}{match_id}?api_key={api_key}')

In [ ]:
def get_challenger_match_hists(region, challengers_df = all_challengers, has_account_id=False):
    challengers_df = challengers_df[['summonerId','region']]
    challengers_df['player_ids'] = [np.nan] * len(challengers_df)
    challengers_df = challengers_df[challengers_df['region'] == region]
    challengers_df.reset_index(drop=True,inplace=True)
    
    if has_account_id==False:
        for i, tuples in enumerate(challengers_df.values):
            if i % 500 == 0:
                print(f'{i} of {len(challengers_df)} account ids')
            summoner_id, region, unused_pid = tuples
            summoner = get_summoner(region_base_url_dict[region],summoner_id).json()
            account_id = summoner['accountId']
            challengers_df.loc[i,'account_ids'] = account_id
            all_challengers.loc[i,'account_ids'] = account_id
            time.sleep(1.2)
        
    challenger_match_hists = pd.DataFrame()
    for i, account_id in enumerate(challengers_df['account_ids']):
        if i % 500 == 0:
            print(f'{i} of {len(challengers_df)} match histories')
        try:
            match_hist = pd.DataFrame(get_match_hist(account_id, region_base_url_dict[challengers_df.loc[i,'region']]).json()['matches'])
            match_hist['region'] = [challengers_df.loc[i,'region']] * len(match_hist)
            match_hist['account_id'] = [account_id] * len(match_hist)
            challenger_match_hists = pd.concat([challenger_match_hists, match_hist])
        except:
            pass
        time.sleep(1.2)
    
    return challenger_match_hists

In [ ]:
def scrape_seeds(region):
    print(f'{region} thread initialized')
    start_time = time.time()
    base_url = region_base_url_dict[region]
    
    matches_list = []
    scraped_ids = set()
    timelines_list = []
    scraped_participants = set()
    unscraped_ids = set()
    scraped_matches = []
    
    with jsonlines.open(f'../data/scraped_ids_{region}.jsonl') as infile:
        print(f'reading scraped_ids_{region}')
        for line in infile.iter():
            scraped_ids.add(line)       
    
    with jsonlines.open(f'../data/unscraped_ids_{region}.jsonl') as infile:
            print(f'reading unscraped_ids_{region}')
            for line in infile.iter():
                unscraped_ids.add(line)
                
    with jsonlines.open(f'../data/matches_{region}.jsonl') as infile:
            print(f'reading matches_{region}')
            for line in infile.iter():
                scraped_matches.append(dict(line)['gameId'])     
    
    seed = pd.read_csv('../data/challenger_match_hists_ranked_only.csv')
    
    seed = seed[seed['region']==region]
    
    seed.reset_index(drop=True,inplace=True)
    
    for i, account_id in enumerate(seed['account_id']):
        if account_id not in scraped_ids or seed.loc[i,'gameId'] not in scraped_matches:
            scraped_ids.add(account_id)
            match, timeline = get_match(seed.loc[i,'gameId'], base_url)
        
            if match.status_code == 200 and timeline.status_code == 200:
                matches_list.append(match.json())
                timelines_list.append(timeline.json())
                [scraped_participants.add(part['player']['accountId']) for part in match.json()['participantIdentities']]
                scraped_matches.append(seed.loc[i,'gameId'])
            else:
                print(f'matches error: {match.status_code}\ntimelines error: {timeline.status_code}\nSummoner: {account_id}\nRegion:{region}')
            time.sleep(2.4)
        
        else:
            pass
        
        if 'match' in locals():
            if i % 100 == 0 or match.status_code == 403:
                print(f'{i} matches scraped of {len(seed)}\nRegion: {region}\n')
                try:
                    with open(f'../data/matches_{region}.jsonl', 'a') as outfile:
                        for entry in matches_list:
                            json.dump(entry, outfile)
                            outfile.write('\n')
                            matches_list = []

                    with open(f'../data/timelines_{region}.jsonl', 'a') as outfile:
                        for entry in timelines_list:
                            json.dump(entry, outfile)
                            outfile.write('\n')
                            timelines_list = []

                    with open(f'../data/scraped_ids_{region}.jsonl', 'w') as outfile:
                        for entry in scraped_ids:
                            json.dump(entry, outfile)
                            outfile.write('\n')

                    unscraped_ids = unscraped_ids.union(scraped_participants)
                    unscraped_ids -= scraped_ids

                    with open(f'../data/unscraped_ids_{region}.jsonl', 'w') as outfile:
                        for entry in unscraped_ids:
                            json.dump(entry, outfile)
                            outfile.write('\n')
                except:
                    pass

                if match.status_code == 403:
                    return None
            
    with open(f'../data/matches_{region}.jsonl', 'a') as outfile:
        for entry in matches_list:
            json.dump(entry, outfile)
            outfile.write('\n')

    with open(f'../data/timelines_{region}.jsonl', 'a') as outfile:
        for entry in timelines_list:
            json.dump(entry, outfile)
            outfile.write('\n')

    with open(f'../data/scraped_ids_{region}.jsonl', 'a') as outfile:
        for entry in scraped_ids:
            json.dump(entry, outfile)
            outfile.write('\n')

    print(f'Seeds scraped, region = {region}')

    unscraped_ids = unscraped_ids.union(scraped_participants)
    unscraped_ids -= scraped_ids
    
    with open(f'../data/unscraped_ids_{region}.jsonl', 'w') as outfile:
        for entry in unscraped_ids:
            json.dump(entry, outfile)
            outfile.write('\n')
    

In [8]:
def remove_duplicates(region):
    match_list = []
    timeline_list = []
    
    with jsonlines.open(f'../data/matches_{region}.jsonl') as infile:
                for line in infile.iter():
                    match_list.append(line) 
                    
    with jsonlines.open(f'../data/timelines_{region}.jsonl') as infile:
            for line in infile.iter():
                    timeline_list.append(line)
    
    del infile
    ranked_matches = []
    ranked_timelines = []
    is_ranked = []
    game_ids = set()
    
    for i, match in enumerate(match_list):
        if match['queueId'] == 420 and match['gameId'] not in game_ids:
            ranked_matches.append(match)
            is_ranked.append(i)
            game_ids.add(match['gameId'])
    del match_list
    
    
    with open(f'../data/cleaned/matches_{region}.jsonl', 'w') as outfile:
        for entry in ranked_matches:
            json.dump(entry, outfile)
            outfile.write('\n')
    del ranked_matches
    
    
    for i, timeline in enumerate(timeline_list):
        if i in is_ranked:
            ranked_timelines.append(timeline)
    del timeline_list
            
    

    with open(f'../data/cleaned/timelines_{region}.jsonl', 'w') as outfile:
        for entry in ranked_timelines:
            json.dump(entry, outfile)
            outfile.write('\n')   
    del ranked_timelines
    del is_ranked
    del game_ids

In [9]:
def remove_short_games(region):
    match_list = []
    timeline_list = []
    
    with jsonlines.open(f'../data/cleaned/matches_{region}.jsonl') as infile:
                for line in infile.iter():
                    match_list.append(line) 
    del infile
                 
    with jsonlines.open(f'../data/cleaned/timelines_{region}.jsonl') as infile:
                for line in infile.iter():
                    timeline_list.append(line) 
    del infile
                    
    good_matches = []
    good_timelines = []
    valid = []
    game_ids = set()
    
    for i, match in enumerate(match_list):
        if match['gameDuration'] / 60 > 15 and match['gameId'] not in game_ids:
            add = True
            # Remove games where a player didn't reach level 6 (based on EDA)
            for participant in match['participants']:
                if participant['stats']['champLevel'] < 7:
                    add = False
            if add:
                good_matches.append(match)
                valid.append(i)
                game_ids.add(match['gameId'])
            else:
                pass
    
    del match_list
  
    with open(f'../data/cleaned/matches_{region}.jsonl', 'w') as outfile:
        for entry in good_matches:
            json.dump(entry, outfile)
            outfile.write('\n')
    del good_matches
    
    for i, timeline in enumerate(timeline_list):
        if i in valid:
            good_timelines.append(timeline)
    
    del timeline_list
            
    

    with open(f'../data/cleaned/timelines_{region}.jsonl', 'w') as outfile:
        for entry in good_timelines:
            json.dump(entry, outfile)
            outfile.write('\n')
            
    del good_timelines
    del valid
    del game_ids

In [ ]:
#  import threading

#  for region in region_base_url_dict.keys():
#      some_thread = threading.Thread(target = scrape_seeds, args=[region])
#      some_thread.start()

In [ ]:
scraped_matches = []
scraped_ids = set()

with jsonlines.open(f'../data/scraped_ids_na.jsonl') as infile:
        print(f'reading scraped_ids_na')
        for line in infile.iter():
            scraped_ids.add(line)

with jsonlines.open(f'../data/matches_na.jsonl') as infile:
            print(f'reading matches_na')
            for line in infile.iter():
                scraped_matches.append(dict(line)['gameId'])
                
na_hists = challenger_match_hists[challenger_match_hists['region'] == 'na']

for i in range(1,len(na_hists)):
    print(na_hists.loc[i,'gameId'] not in scraped_matches or na_hists.loc[i,'account_id'] not in scraped_ids)

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers = 11) as executor:
    executor.map(scrape_seeds, region_base_url_dict.keys())

In [ ]:
all_challengers = get_all_challengers()

In [ ]:
all_challengers.reset_index(drop=True,inplace=True)

In [ ]:
Spawwwwn = get_summoner(region_base_url_dict[all_challengers['region'][0]], all_challengers['summonerId'][0])

In [ ]:
Spawwwwn.json()

In [ ]:
all_challengers.head()

In [ ]:
all_challengers.loc[0,['summonerId','region']]

In [ ]:
test = get_match_hist(Spawwwwn.json()['accountId'],region_base_url_dict[all_challengers['region'][0]])

In [ ]:
test.json().keys()

In [ ]:
#pd.DataFrame(test.json()['matches'])

In [ ]:
%%time
#challenger_match_hists = get_challenger_match_hists(all_challengers)

match_hist_list = []
with concurrent.futures.ThreadPoolExecutor(max_workers = 11) as executor:
    for result in executor.map(get_challenger_match_hists, region_base_url_dict.keys()):
        match_hist_list.append(result)

In [ ]:
challenger_match_hists = pd.concat(match_hist_list)

In [ ]:
challenger_match_hists.reset_index(drop=True,inplace=True)

In [ ]:
challenger_match_hists.to_csv('../data/challenger_match_hists_ranked_only.csv', index=False)

In [ ]:
for region in list(region_base_url_dict.keys())[6:]:
    remove_duplicates(region)

In [ ]:
for region in region_base_url_dict.keys():
    remove_short_games(region)

In [10]:
remove_duplicates('la1')
remove_short_games('la1')

In [43]:
from urllib.parse import quote
x = get_summoner_by_name(region_base_url_dict['na'],quote('smurfelise'))

In [57]:
quote('시간은또금세흘러')

'%EC%8B%9C%EA%B0%84%EC%9D%80%EB%98%90%EA%B8%88%EC%84%B8%ED%9D%98%EB%9F%AC'

In [44]:
x.json()

{'id': 'JA9PzHteIJvObXbm_ltkED-zJ_cZjCrw-BObnVal_WIcT3Q',
 'accountId': 'iGvBI4ldeWFC0hRVHkwYEBj1jBp8Fnj9DTPn4izvk4EYmw',
 'puuid': 'fOJFYvs6bCz4Nq86mHciFiY49orLS0Hpf-tMNimlnywGb01pevchp_jm5ev_c44jGqIKJ8iF0v26-g',
 'name': 'Smurf Elise',
 'profileIconId': 4087,
 'revisionDate': 1596386469000,
 'summonerLevel': 81}

In [45]:
y = get_match_hist(x.json()['accountId'],region_base_url_dict['na'])

In [61]:
y.headers['X-App-Rate-Limit']

'20:1,100:120'

In [62]:
y.headers['X-App-Rate-Limit-Count']

'1:1,2:120'

In [18]:
y.json()['matches']

[{'platformId': 'NA1',
  'gameId': 3555268224,
  'champion': 22,
  'queue': 420,
  'season': 13,
  'timestamp': 1598760541047,
  'role': 'DUO_CARRY',
  'lane': 'BOTTOM'},
 {'platformId': 'NA1',
  'gameId': 3555272721,
  'champion': 22,
  'queue': 420,
  'season': 13,
  'timestamp': 1598758858106,
  'role': 'DUO_CARRY',
  'lane': 'BOTTOM'},
 {'platformId': 'NA1',
  'gameId': 3554661811,
  'champion': 51,
  'queue': 420,
  'season': 13,
  'timestamp': 1598716168136,
  'role': 'DUO_CARRY',
  'lane': 'BOTTOM'},
 {'platformId': 'NA1',
  'gameId': 3553508110,
  'champion': 22,
  'queue': 420,
  'season': 13,
  'timestamp': 1598652416792,
  'role': 'DUO',
  'lane': 'NONE'},
 {'platformId': 'NA1',
  'gameId': 3553499972,
  'champion': 22,
  'queue': 420,
  'season': 13,
  'timestamp': 1598648205468,
  'role': 'DUO_CARRY',
  'lane': 'BOTTOM'},
 {'platformId': 'NA1',
  'gameId': 3553436821,
  'champion': 22,
  'queue': 420,
  'season': 13,
  'timestamp': 1598646584491,
  'role': 'DUO_CARRY',
  '

In [46]:
z = []
zz = []

for match in y.json()['matches']:
    a,b = get_match(match['gameId'],region_base_url_dict['na'])
    if a.status_code == 200 and b.status_code == 200:
        z.append(a.json())
        zz.append(b.json())
    time.sleep(1.2)
        


In [21]:
len(z)

93

In [22]:
len(zz)

93

In [3]:
import pandas as pd
pd.read_csv('../data/role_models_3.csv').columns

Index(['csd_10', 'gold_d_10', 'xpd_10', 'dmg_share', 'dmg_taken_share',
       'vision_score', 'kill_participation', 'obj_dmg_share', 'dragons',
       'barons', 'wards_cleared', 'vision_wards_purchased', 'kda_early',
       'kda_mid', 'kda_late', 'solo_kills', 'teamfight_kills',
       'skirmish_kills', 'wards_early', 'wards_mid', 'wards_late',
       'summoner_id', 'most_played_champ', 'most_played_role',
       'most_played_lane', 'region', 'player_name', 'op_gg',
       'most_played_champ_name', 'role'],
      dtype='object')

In [25]:
def remove_short_games_2(region, match_list, timeline_list):
    
                    
    good_matches = []
    good_timelines = []
    valid = []
    game_ids = set()
    
    for i, match in enumerate(match_list):
        if match['gameDuration'] / 60 > 15 and match['gameId'] not in game_ids:
            add = True
            # Remove games where a player didn't reach level 6 (based on EDA)
            for participant in match['participants']:
                if participant['stats']['champLevel'] < 7:
                    add = False
            if add:
                good_matches.append(match)
                valid.append(i)
                game_ids.add(match['gameId'])
            else:
                pass
    
    del match_list
    
    for i, timeline in enumerate(timeline_list):
        if i in valid:
            good_timelines.append(timeline)
    
    del timeline_list
    del valid
    del game_ids
    
    return good_matches, good_timelines

In [47]:
e, ee = remove_short_games_2('na',z,zz)

In [48]:
f = get_stats(x.json()['id'], 'na', e, ee)

In [30]:
f

,csd_10,gold_d_10,xpd_10,dmg_share,dmg_taken_share,vision_score,kill_participation,obj_dmg_share,dragons,barons,wards_cleared,vision_wards_purchased,kda_early,kda_mid,kda_late,solo_kills,teamfight_kills,skirmish_kills,wards_early,wards_mid,wards_late,summoner_id,most_played_champ,most_played_role,most_played_lane,region,player_name
0,5.643678,264.174713,310.172414,0.2146,0.155079,17.83908,0.44436,0.221058,0.409086,0.164751,0.190992,0.158742,0.160558,0.375789,0.397365,0.079227,0.369459,0.287378,0.237528,0.361634,0.257208,NmMlQYyDZuTiCKPfMtsMyEOWz0YB0jHvxLpIjH_p_-n1v7g,22,DUO_CARRY,BOTTOM,na,yamino hikari


In [49]:
f.to_csv('../data/morgan_test.csv', index=False)

In [23]:
def get_stats(summoner, region, matches, timelines, team_participants = ([1,2,3,4,5],[6,7,8,9,10])):
# For each player:
    output = pd.DataFrame(columns=['champ_played', 'role','lane', 'csd_10',
                                  'gold_d_10','xpd_10','dmg_share','dmg_taken_share','vision_score',
                                  'kill_participation','obj_dmg_share','dragons','barons','wards_cleared',
                                  'vision_wards_purchased','kda_early', 'kda_mid', 'kda_late', 
                                   'solo_kills', 'teamfight_kills', 'skirmish_kills', 'wards_early', 
                                   'wards_mid', 'wards_late'])
    # Main role = most played role where role is played > 40% of the time
    # Main champion = most played champion
    # Select all matches and timelines that they are in
    for i, match in enumerate(matches):
        is_in_game = summoner in [summoner['player']['summonerId'] for summoner in match['participantIdentities']]
        if is_in_game:
        # For each match:
        # Match info:
            game_id = match['gameId']
            player_id = int([player['participantId'] for player in match['participantIdentities'] if player['player']['summonerId'] == summoner][0]) - 1
            player_name = match['participantIdentities'][player_id]['player']['summonerName']
            
            # Identify champion played
            champ_played = match['participants'][player_id]['championId']
            
            # Identify role played
            role = match['participants'][player_id]['timeline']['role']
            lane = match['participants'][player_id]['timeline']['lane']
            # Identify lane opponent
            team = match['participants'][player_id]['teamId']
            team_index = int(str(team)[0]) - 1
            teammates = team_participants[team_index]
               
            # Get CSD, GoldD and XPD @ 10
            csd_10 = match['participants'][player_id]['timeline']['creepsPerMinDeltas']['0-10']
            gold_d_10 = match['participants'][player_id]['timeline']['goldPerMinDeltas']['0-10']
            xpd_10 = match['participants'][player_id]['timeline']['xpPerMinDeltas']['0-10']
            
            # Get DMG share
            total_dmg = np.sum([player['stats']['totalDamageDealtToChampions'] for player in match['participants'] if player['teamId'] == team])
            dmg_share = match['participants'][player_id]['stats']['totalDamageDealtToChampions'] / total_dmg
            
            # Get DMG Taken share
            total_dmg_taken = np.sum([player['stats']['totalDamageTaken'] for player in match['participants'] if player['teamId'] == team])
            dmg_taken_share = match['participants'][player_id]['stats']['totalDamageTaken'] / total_dmg_taken
            
            # Get vision score
            vision_score = match['participants'][player_id]['stats']['visionScore']
            
            # Get overall kill participation
            team_kills = np.sum([player['stats']['kills'] for player in match['participants'] if player['participantId'] in teammates]) + 1
            kill_participation = (match['participants'][player_id]['stats']['kills'] + match['participants'][player_id]['stats']['assists']) / team_kills
            
            # % of team's objective damage % of team's turret damage
            total_obj_dmg = np.sum([player['stats']['damageDealtToObjectives'] for player in match['participants'] if player['teamId'] == team])
            obj_dmg_share = match['participants'][player_id]['stats']['damageDealtToObjectives'] / total_obj_dmg
            
            # Team % of dragons killed
            team_dragons = match['teams'][team_index]['dragonKills']
            total_dragons = team_dragons + match['teams'][0 if team_index == 1 else 1]['dragonKills']
            dragons = team_dragons / (total_dragons + 1)
            
            # Team % of barons killed
            team_barons = match['teams'][team_index]['baronKills']
            total_barons = team_barons + match['teams'][0 if team_index == 1 else 1]['baronKills']
            barons = team_barons / (total_barons + 1)
            
            # Get wards cleared
            wards_cleared = match['participants'][player_id]['stats']['wardsKilled']
            
            # Get pinks purchased
            vision_wards_purchased = match['participants'][player_id]['stats']['visionWardsBoughtInGame']
            
            # For each timeline:
            timeline = timelines[i]
            
            kills = 0
            kda = 0
            kda_early = 0
            kda_mid = 0
            kda_late = 0
            solo_kills = 0
            teamfight_kills = 0
            skirmish_kills = 0
            wards = 0
            wards_early = 0
            wards_mid = 0
            wards_late = 0
            
            for i, frame in enumerate(timeline['frames']):
                for event in frame['events']:
                    # Get 0-10 K+D+A
                    # Get 10-20 K+D+A
                    # Get 20+ K+D+A
                    if event['type'] == 'CHAMPION_KILL':
                        if (player_id + 1) == event['killerId'] or (player_id + 1) == event['victimId'] or (player_id + 1) in event['assistingParticipantIds']:
                            kda += 1
                            if i < 12:
                                kda_early += 1
                            elif i < 22:
                                kda_mid += 1
                            else:
                                kda_late += 1

                        if (player_id + 1) == event['killerId']:
                            kills += 1
                    # get number of solo kills
                    # get number of skirmish kills
                    # get number of teamfight kills
                            if event['assistingParticipantIds'] == []:
                                solo_kills += 1
                            if len(event['assistingParticipantIds']) == 1:
                                skirmish_kills += 1
                            if  len(event['assistingParticipantIds']) > 1:
                                teamfight_kills += 1
                    # Get 0-10 wards placed
                    # Get 10-20 wards placed
                    # Get 20+ wards placed
                    if event['type'] == 'WARD_PLACED' and (player_id + 1) == event['creatorId']:
                        wards += 1
                        if i < 12:
                            wards_early += 1
                        elif i < 22:
                            wards_mid += 1
                        else:
                            wards_late += 1 
            
            solo_kills /= kills + 1
            skirmish_kills /= kills + 1
            teamfight_kills /= kills + 1
            wards_early /= wards + 1
            wards_mid /= wards + 1
            wards_late /= wards + 1 
            kda_early /= kda + 1
            kda_mid /= kda + 1
            kda_late /= kda + 1
            
            # Wards cleared and vision wards purchased as a % of wards placed
            wards_cleared /= wards + 1
            vision_wards_purchased /= wards + 1
            
            output = output.append({'champ_played':champ_played, 'role':role, 'lane':lane,
             'csd_10':csd_10,'gold_d_10':gold_d_10,'xpd_10':xpd_10,'dmg_share':dmg_share,
             'dmg_taken_share':dmg_taken_share,'vision_score':vision_score,
             'kill_participation':kill_participation,'obj_dmg_share':obj_dmg_share,'dragons':dragons,
             'barons':barons,'wards_cleared':wards_cleared,'vision_wards_purchased':vision_wards_purchased,
             'kda_early':kda_early, 'kda_mid':kda_mid, 'kda_late':kda_late, 'solo_kills':solo_kills, 
             'teamfight_kills':teamfight_kills, 'skirmish_kills':skirmish_kills, 'wards_early':wards_early, 
             'wards_mid':wards_mid, 'wards_late':wards_late}, ignore_index=True)
    
    most_played_champ = output['champ_played'].value_counts().index[0]
    most_played_role = output['role'].value_counts().index[0]
    most_played_lane = output['lane'].value_counts().index[0]
    
    output.drop(columns=['champ_played','role','lane'], inplace=True)

    output = pd.DataFrame(output.mean()).transpose()
    output['summoner_id'] = [summoner]
    output['most_played_champ'] = most_played_champ
    output['most_played_role'] = most_played_role
    output['most_played_lane'] = most_played_lane
    output['region'] = region
    output['player_name'] = player_name
    return output